In [6]:
import cv2
import time
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from PIL import ImageGrab

### Кадрируем видео и решейпим кадры

Из-за ограничений по памяти при отправке тестового я не смогла оставить папку data

In [6]:
'''
В данном видео 15 фпс, поэтому берем кадр ровно раз в секунду 
'''

vidcap = cv2.VideoCapture('data/camera.mp4') 
def getFrame(sec): 
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000) 
    hasFrames,image = vidcap.read() 
    if hasFrames: 
        cv2.imwrite("data/frames/frame%d.jpg" % sec, image)   
    return hasFrames 
sec = 0 
frameRate = 1
success = getFrame(sec) 
while success: 
    sec = sec + frameRate 
    success = getFrame(sec) 

In [4]:
for img in range(0, 37682):
    resim = cv2.imread("data/frames/frame%d.jpg" % img)    
    crop_img = resim[210:445, 115:350]
    crop_img = cv2.resize(crop_img, (116, 116))
    cv2.imwrite("data/crop_img/crop%d.jpg" % img, crop_img)

In [36]:
splitfolders.ratio('data/dataset', output="dataset_splited", seed=1337, ratio=(.8, .2)) 

### Строим модель на Keras

In [3]:
import splitfolders
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras import backend
import random
import scipy
from keras.utils import load_img
from keras.utils import img_to_array
from keras.models import load_model

In [4]:
train_data_dir = 'dataset_splited/train'
validation_data_dir = 'dataset_splited/val'
epochs = 20
batch_size = 20
img_width, img_height = 116, 116

if backend.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3) #116,116,3

In [5]:
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

Found 30123 images belonging to 2 classes.
Found 7532 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(lr=0.001),
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 115, 115, 32)      416       
                                                                 
 activation (Activation)     (None, 115, 115, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 57, 57, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 32)        4128      
                                                                 
 activation_1 (Activation)   (None, 56, 56, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                    

c:\Users\Sveta\anaconda3\envs\cv_env\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [8]:
model.fit(
	train_generator,
	steps_per_epoch=100,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=50,
	verbose=-1)

model.evaluate(validation_generator)

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
377/377 [==============================] - 21s 56ms/step - loss: 0.0252 - accuracy: 0.9942


[0.025204481557011604, 0.9941582679748535]

In [10]:
model.save("model.h5")

### Тестируем модель

In [8]:
model = load_model('model.h5')

def prediction(path):
    image = load_img(path, target_size=(116, 116))
    img_pred = img_to_array(image)
    img_pred = np.expand_dims(img_pred, axis=0)
    rslt = model.predict(img_pred)

    if rslt[0][0] == 1:
        prediction = 'рабочее место пустует'
    else:
        prediction = 'человек на рабочим месте'

    print(prediction)

In [9]:
prediction(r'C:\Users\Sveta\projects\cv\dataset_splited\val\human\crop12936.jpg')

1/1 [==============================] - 0s 186ms/step
человек на рабочим месте


In [11]:
prediction(r"C:\Users\Sveta\projects\cv\dataset_splited\val\no_human\crop718.jpg")

1/1 [==============================] - 0s 40ms/step
рабочее место пустует


In [10]:
prediction(r'C:\Users\Sveta\projects\cv\dataset_splited\val\human\crop22928.jpg')

1/1 [==============================] - 0s 40ms/step
человек на рабочим месте
